# pyfoamalgo azimuthal integration benchmark

In [ ]:
import os.path as osp

import numpy as np
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator as PyfaiAzimuthalIntegrator
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

import pyfoamalgo
print(pyfoamalgo.__version__)

from pyfoamalgo import AzimuthalIntegrator, ConcentricRingsFinder
from pyfoamalgo import mask_image_data

In [ ]:
def create_image(w, h, cx, cy, *, aspect_ratio=1, lw=2, radius=None):
    if cx is None:
        cx = int(w / 2)
    if cy is None:
        cy = int(h / 2)
        
    img = np.zeros((w, h), dtype=np.float32)
    
    if radius is None:
        radius = [20, 100, 130, 200, 300]
    
    for r in radius:
        for theta in np.linspace(0, 360, 10000):
            y = cy + aspect_ratio * r * np.cos(theta) + (2 * np.random.random_sample() - 1.)
            x = cx + r * np.sin(theta) + (2 * np.random.random_sample() - 1.)
            img[int(y-lw/2):int(y+lw/2), int(x-lw/2):int(x+lw/2)] = 1
        
    
    img[:, 100:110] = np.nan
    img[int(h/2):int(h/2) + 10, :] = np.nan
    
    return img

In [ ]:
cy, cx = 400, 320
pixel1, pixel2 = 200e-6, 100e-6  # pixel size (y, x)

img = create_image(480, 640, cx, cy, aspect_ratio=pixel2/pixel1)

_, ax = plt.subplots(figsize=(12, 12))
ax.imshow(img)

#### Integrate a single image

In [ ]:
dist = 1  # sample distance
npt = 512  # number of integration points
poni1, poni2 = cy * pixel1, cx * pixel2  # integration center (y, x)

In [ ]:
# %%timeit

pyfai_method = 'nosplit_csr'
pyfai_integrator = PyfaiAzimuthalIntegrator(
    dist=dist, poni1=poni1, poni2=poni2, pixel1=pixel1, pixel2=pixel2, wavelength=1e-10)

q_gt, I_gt = pyfai_integrator.integrate1d(img, npt, unit="q_A^-1", method=pyfai_method)

In [ ]:
%timeit q_gt, I_gt = pyfai_integrator.integrate1d(img, npt, unit="q_A^-1", method=pyfai_method)

In [ ]:
# %%timeit

integrator = AzimuthalIntegrator(
    dist=dist, poni1=poni1, poni2=poni2, pixel1=pixel1, pixel2=pixel2, wavelength=1e-10)

q, I = integrator.integrate1d(img, npt=npt)

In [ ]:
%timeit q, I = integrator.integrate1d(img, npt=npt)

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))

ax.plot(1e-10 * q, I, '-', label='foamalgo')
ax.plot(q_gt, I_gt, '--', label='pyFAI')
ax.set_xlabel("q (1/A)", fontsize=16)
ax.set_ylabel("I (arb.)", fontsize=16)
ax.legend(fontsize=16)

#### Integrate an array of images

In [ ]:
import multiprocessing as mp

print(mp.cpu_count())

In [ ]:
img_array = np.tile(img, (40, 1, 1))

q_a, I_a = integrator.integrate1d(img_array, npt=npt)
np.testing.assert_array_equal(q_a, q)
np.testing.assert_array_equal(I_a[0], I)
np.testing.assert_array_equal(I_a[39], I)

%timeit integrator.integrate1d(img_array, npt=npt)

#### Concentric ring finding

In [ ]:
cx0 = cx - 8
cy0 = cy + 8
finder = ConcentricRingsFinder(pixel2, pixel1)
cx_opt, cy_opt = finder.search(img, cx0, cy0, min_count=1)

print("Optimized cx = ", cx_opt, ", cy = ", cy_opt)
print("Ground truth cx = ", cx, ", cy = ", cy)

In [ ]:
%timeit finder.search(img, cx, cy, min_count=1)